In [ ]:
import pandas as pd
import re
climate_data = pd.read_csv("climate_streaming.csv")
climate_data = climate_data.sample(frac=1).reset_index(drop=True)

import pymongo
from pymongo import MongoClient
from datetime import datetime, timedelta
import random



def getNextClimateDay():
    #gets the date of the latest climate record
    client = MongoClient('192.168.0.108', 27017) #change to local ip
    db = client.fit3182_db 
    climate = db["climate"]
    hotspots = db['hotspots']
    latest_record = climate.find_one({}, sort=[('date', pymongo.DESCENDING)])
    latest_date = latest_record['date']
    client.close()
    return latest_date


latest_date = getNextClimateDay()

import pandas as pd
import random
import time
from kafka3 import KafkaProducer
import datetime as dt
from time import sleep
from json import dumps

hostip = "192.168.0.108" # change to local ip

def pushData(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(value))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                 api_version=(0,10))
        
    except Exception as ex:
        print('could not connect to Kafka')
        print(str(ex))
    finally:
        return _producer
        
        
if __name__ == '__main__':
    topic = 'Producer1'
    
    print('starting kafka stream')
    
    producer = connect_kafka_producer()
    
    #randomly order the climate data
    climate_data = climate_data.sample(frac=1).reset_index(drop=True)
    
    for index, row in climate_data.iterrows():
        df = row
        latest_date = (latest_date + timedelta(days=1)) #set the next day to the latest date
        #append producer info
        df['created_on'] = latest_date.isoformat() #datetime of when data is created (would be the current datetime in real life scenario)
        df['producer'] = topic #used to identify which producer data came from
        record_json = df.to_json()
        pushData(producer,topic, 'parsed', record_json)
        sleep(10) #simulate day wait period
    
        


starting kafka stream
Message published successfully. Data: {"latitude":-36.0459,"longitude":143.8907,"air_temperature_celcius":21,"relative_humidity":58.7,"windspeed_knots":7.8,"max_wind_speed":13.0,"precipitation ":" 0.00I","GHI_w\/m2":167,"created_on":"2024-11-22T00:00:00","producer":"Producer1"}
Message published successfully. Data: {"latitude":-37.213,"longitude":142.941,"air_temperature_celcius":23,"relative_humidity":61.9,"windspeed_knots":8.0,"max_wind_speed":15.9,"precipitation ":" 0.00I","GHI_w\/m2":178,"created_on":"2024-11-23T00:00:00","producer":"Producer1"}
Message published successfully. Data: {"latitude":-37.0667,"longitude":143.0582,"air_temperature_celcius":11,"relative_humidity":39.7,"windspeed_knots":11.8,"max_wind_speed":19.0,"precipitation ":" 0.28G","GHI_w\/m2":102,"created_on":"2024-11-24T00:00:00","producer":"Producer1"}
Message published successfully. Data: {"latitude":-37.367,"longitude":148.04,"air_temperature_celcius":12,"relative_humidity":47.6,"windspeed_